In [1]:
!python --version

Python 3.7.5


In [2]:
import tensorflow as tf
print(tf.__version__)

1.15.0


In [12]:
from bert import extract_features
from bert import tokenization
from bert import modeling

In [4]:
VOCAB_FILE_LOCATION='../uncased_L-12_H-768_A-12/vocab.txt'

tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE_LOCATION)

In [5]:
examples = extract_features.read_examples('./data.txt')

In [68]:
features = extract_features.convert_examples_to_features(examples=examples, seq_length=25, tokenizer=tokenizer)

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] the quick brown fox jumps over the lazy dog . [SEP]
INFO:tensorflow:input_ids: 101 1996 4248 2829 4419 14523 2058 1996 13971 3899 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [28]:
print(len(features))

print(features[0].tokens)

1
['[CLS]', 'the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.', '[SEP]']


In [34]:
unique_id_to_feature = {}
for feature in features:
    unique_id_to_feature[feature.unique_id] = feature

In [13]:
model_fn = extract_features.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file('../uncased_L-12_H-768_A-12/bert_config.json'),
    init_checkpoint='../uncased_L-12_H-768_A-12/bert_model.ckpt',
    layer_indexes=[-1, -2, -3, -4],
    use_tpu=False,
    use_one_hot_embeddings=False
)

In [14]:
run_config = tf.contrib.tpu.RunConfig(
    master=None,
    tpu_config=tf.contrib.tpu.TPUConfig(
        num_shards=8,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
    )
)

In [15]:
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False,
    model_fn=model_fn,
    predict_batch_size=32,
    config=run_config
)

INFO:tensorflow:Using config: {'_model_dir': '/var/folders/1h/kbc1m98s7434dpfydry4x8140000gn/T/tmpwsdx4ad7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1304b3190>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_

In [77]:
input_fn = extract_features.input_fn_builder(
    features=features,
    seq_length=25 # Has to be at least sequence length of input used in `convert_examples_to_features`.
)

In [78]:
for result in estimator.predict(input_fn, yield_single_examples=True):
    print(result.keys())
    output = result

INFO:tensorflow:Could not find trained model in model_dir: /var/folders/1h/kbc1m98s7434dpfydry4x8140000gn/T/tmpwsdx4ad7, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  n

In [32]:
output['unique_id']

0

In [79]:
unique_id_to_feature[0].tokens

['[CLS]',
 'the',
 'quick',
 'brown',
 'fox',
 'jumps',
 'over',
 'the',
 'lazy',
 'dog',
 '.',
 '[SEP]']

In [80]:
output['layer_output_0'].shape

(25, 768)

In [81]:
the1 = output['layer_output_0'][1]
the5 = output['layer_output_0'][5]
the7 = output['layer_output_0'][7]

In [82]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b)/(np.linalg.norm(a) * np.linalg.norm(b))

In [83]:
cosine_similarity(the1, the7)

0.82807493

In [84]:
cosine_similarity(the1, the5)

0.53417337

In [85]:
cosine_similarity(the5, the7)

0.5922921